In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

In [5]:
sys.path.insert(0, '../code/')

In [33]:
import torch
import torch.nn as nn

from config import cfg, cfg_from_file, _merge_a_into_b
from mac import MACNetwork
from utils import load_vocab
from datasets import ClevrScenesDataset, scenes_collate_fn

import yaml
from torchsummaryX import summary
from easydict import EasyDict as edict

In [25]:
config_file = \
'''
GPU_ID: '-1'
WORKERS: 1
CUDA: False

TRAIN:
    FLAG: True
    LEARNING_RATE: 0.0001
    BATCH_SIZE: 64
    MAX_EPOCHS: 25
    SNAPSHOT_INTERVAL: 5
    WEIGHT_INIT: "xavier_uniform"
    EALRY_STOPPING: True
    PATIENCE: 5
    CLIP_GRADS: True
    RADAM: False
    # MAX_STEPS: 4

DATASET:
    DATA_DIR: '/Users/sebamenabar/Documents/datasets/CLEVR/CLEVR_v1.0/data'

model:
    num_lobs: 0
    max_step: 12
    separate_syntax_semantics: False
    common:
        module_dim: 512
    input_unit:
        wordvec_dim: 300
        rnn_dim: 512
        bidirectional: True
        separate_syntax_semantics_embeddings: True
        stem_act: 'ELU'
    control_unit:
        control_feed_prev: True
        control_cont_activation: 'TANH'
    # read_unit:
    write_unit:
        rtom: False
        self_attn: True
        gate: True
        gate_shared: False
    # output_unit: 
'''

In [29]:
yaml_cfg = edict(yaml.safe_load(config_file))
_merge_a_into_b(yaml_cfg, cfg)

vocab = load_vocab(cfg)

In [18]:
ds = ClevrScenesDataset(
    '/Users/sebamenabar/Documents/datasets/CLEVR/CLEVR_v1.0/data/',
    '/Users/sebamenabar/Documents/datasets/CLEVR/CLEVR_v1.0/scenes/',
    'val',
)

In [34]:
loader = torch.utils.data.DataLoader(ds, batch_size=64, num_workers=0, shuffle=True, collate_fn=scenes_collate_fn)
b = next(iter(loader))

cfg.model.num_lobs = 0
cfg.model.max_step = 12
model = MACNetwork(cfg=cfg, vocab=vocab)
summary(model, b['scenes'], b['question'], b['question_length'], b['scene_length'])

/usr/local/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


                                          Kernel Shape   Output Shape  \
Layer                                                                   
0_input_unit.Embedding_emb_color              [128, 8]     [402, 128]   
1_input_unit.Embedding_emb_material           [128, 2]     [402, 128]   
2_input_unit.Embedding_emb_shape              [128, 3]     [402, 128]   
3_input_unit.Embedding_emb_size               [128, 2]     [402, 128]   
4_input_unit.Linear_attributes              [515, 512]     [402, 512]   
5_input_unit.Embedding_encoder_embed         [300, 90]  [64, 35, 300]   
6_input_unit.Dropout_embedding_dropout               -  [64, 35, 300]   
7_input_unit.LSTM_encoder                            -    [1181, 512]   
8_input_unit.Dropout_question_dropout                -      [64, 512]   
9_mac.control.control_input.Linear_0        [512, 512]      [64, 512]   
10_mac.control.control_input.Tanh_1                  -      [64, 512]   
11_mac.control.control_input_u.Linear_0     [512, 5

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_input_unit.Embedding_emb_color,"[128, 8]","[402, 128]",1024.0,1024.0
1_input_unit.Embedding_emb_material,"[128, 2]","[402, 128]",256.0,256.0
2_input_unit.Embedding_emb_shape,"[128, 3]","[402, 128]",384.0,384.0
3_input_unit.Embedding_emb_size,"[128, 2]","[402, 128]",256.0,256.0
4_input_unit.Linear_attributes,"[515, 512]","[402, 512]",264192.0,263680.0
5_input_unit.Embedding_encoder_embed,"[300, 90]","[64, 35, 300]",27000.0,27000.0
6_input_unit.Dropout_embedding_dropout,-,"[64, 35, 300]",NaN,NaN
7_input_unit.LSTM_encoder,-,"[1181, 512]",1142784.0,1138688.0
8_input_unit.Dropout_question_dropout,-,"[64, 512]",NaN,NaN
